In [1]:
using DataValues, BenchmarkTools, DataFrames

# Union{Missing,T} problems

This notebook tries to present a concise problem case for the `Union{T,Missing}` story that hits packages like [Query.jl](https://github.com/queryverse/Query.jl). It highlights three distinct issues. I'm not sure that these are the only ones, but I do know that these are quite fundamental for something like [Query.jl](https://github.com/queryverse/Query.jl).

I will use `DataFrame` in a couple of places to visualize data, other than that it doesn't play a role here, i.e. I only use it to generate a slightly easier output in the notebook.

## Source data sets

The issue here is about iterators of named tuples with fields that can have missing values. I'm using vectors of these named tuples as the source for the simple example, but any solution would have to work not just for arrays but any iterator of named tuples.

The source data has six columns and four rows. I'm using six columns because I think things might be easier for very small column numbers (not sure), so I want to make sure the test case is not too easy.

The source data that uses `Missing` is this here:

In [2]:
A = NamedTuple{(:a,:b,:c,:d,:e,:f),Tuple{Union{Int,Missing},Union{Float64,Missing},Union{Int,Missing},Union{Float64,Missing},Union{Int,Missing},Union{Float64,Missing}}}[
    (a=1,b=2.,c=missing,d=4,e=missing,f=5),
    (a=missing, b=3.,c=3,d=4,e=4,f=5),
    (a=8, b=missing,c=3,d=missing,e=4,f=5),
    (a=missing,b=missing,c=3,d=4,e=4,f=5)
]

4-element Array{NamedTuple{(:a, :b, :c, :d, :e, :f),Tuple{Union{Missing, Int64},Union{Missing, Float64},Union{Missing, Int64},Union{Missing, Float64},Union{Missing, Int64},Union{Missing, Float64}}},1}:
 NamedTuple{(:a, :b, :c, :d, :e, :f),Tuple{Union{Missing, Int64},Union{Missing, Float64},Union{Missing, Int64},Union{Missing, Float64},Union{Missing, Int64},Union{Missing, Float64}}}((1, 2.0, missing, 4.0, missing, 5.0))
 NamedTuple{(:a, :b, :c, :d, :e, :f),Tuple{Union{Missing, Int64},Union{Missing, Float64},Union{Missing, Int64},Union{Missing, Float64},Union{Missing, Int64},Union{Missing, Float64}}}((missing, 3.0, 3, 4.0, 4, 5.0))      
 NamedTuple{(:a, :b, :c, :d, :e, :f),Tuple{Union{Missing, Int64},Union{Missing, Float64},Union{Missing, Int64},Union{Missing, Float64},Union{Missing, Int64},Union{Missing, Float64}}}((8, missing, 3, missing, 4, 5.0))    
 NamedTuple{(:a, :b, :c, :d, :e, :f),Tuple{Union{Missing, Int64},Union{Missing, Float64},Union{Missing, Int64},Union{Missing, Float64},

It is probably easier to look at it when it is formatted as a table, I'm using `DataFrame`s for that nicer output:

In [3]:
DataFrame(A)

,a,b,c,d,e,f
,Int64⍰,Float64⍰,Int64⍰,Float64⍰,Int64⍰,Float64⍰
1,1,2.0,missing,4.0,missing,5.0
2,missing,3.0,3,4.0,4,5.0
3,8,missing,3,missing,4,5.0
4,missing,missing,3,4.0,4,5.0


The source data for the `DataValue` case is this here:

In [4]:
B = [
    (a=DataValue(1),b=DataValue(2.),c=DataValue{Int}(),d=DataValue(4.),e=DataValue{Int}(),f=DataValue(5.)),
    (a=DataValue{Int}(),b=DataValue(3.),c=DataValue(3),d=DataValue(4.),e=DataValue(4),f=DataValue(5.)),
    (a=DataValue(8),b=DataValue{Float64}(),c=DataValue(3),d=DataValue{Float64}(),e=DataValue(4),f=DataValue(5.)),
    (a=DataValue{Int}(),b=DataValue{Float64}(),c=DataValue(3),d=DataValue(4.),e=DataValue(4),f=DataValue(5.))
]

4-element Array{NamedTuple{(:a, :b, :c, :d, :e, :f),Tuple{DataValue{Int64},DataValue{Float64},DataValue{Int64},DataValue{Float64},DataValue{Int64},DataValue{Float64}}},1}:
 (a = DataValue{Int64}(1), b = DataValue{Float64}(2.0), c = DataValue{Int64}(), d = DataValue{Float64}(4.0), e = DataValue{Int64}(), f = DataValue{Float64}(5.0)) 
 (a = DataValue{Int64}(), b = DataValue{Float64}(3.0), c = DataValue{Int64}(3), d = DataValue{Float64}(4.0), e = DataValue{Int64}(4), f = DataValue{Float64}(5.0))
 (a = DataValue{Int64}(8), b = DataValue{Float64}(), c = DataValue{Int64}(3), d = DataValue{Float64}(), e = DataValue{Int64}(4), f = DataValue{Float64}(5.0))     
 (a = DataValue{Int64}(), b = DataValue{Float64}(), c = DataValue{Int64}(3), d = DataValue{Float64}(4.0), e = DataValue{Int64}(4), f = DataValue{Float64}(5.0))   

Same thing again as a table for easier viewing:

In [5]:
DataFrame(B)

,a,b,c,d,e,f
,Int64⍰,Float64⍰,Int64⍰,Float64⍰,Int64⍰,Float64⍰
1,1,2.0,missing,4.0,missing,5.0
2,missing,3.0,3,4.0,4,5.0
3,8,missing,3,missing,4,5.0
4,missing,missing,3,4.0,4,5.0


## First order problem: performance

The first test we are running is a simple projection over this that rearranges columns. The code for the `Missing` and `DataValue` case is the same:

In [6]:
A2 = ((b=i.b,c=i.c,d=i.d,e=i.e,f=i.f,a=i.a) for i in A) |> collect

4-element Array{NamedTuple{(:b, :c, :d, :e, :f, :a),T} where T<:Tuple,1}:
 (b = 2.0, c = missing, d = 4.0, e = missing, f = 5.0, a = 1)
 (b = 3.0, c = 3, d = 4.0, e = 4, f = 5.0, a = missing)      
 (b = missing, c = 3, d = missing, e = 4, f = 5.0, a = 8)    
 (b = missing, c = 3, d = 4.0, e = 4, f = 5.0, a = missing)  

In [7]:
B2 = ((b=i.b,c=i.c,d=i.d,e=i.e,f=i.f,a=i.a) for i in B) |> collect

4-element Array{NamedTuple{(:b, :c, :d, :e, :f, :a),Tuple{DataValue{Float64},DataValue{Int64},DataValue{Float64},DataValue{Int64},DataValue{Float64},DataValue{Int64}}},1}:
 (b = DataValue{Float64}(2.0), c = DataValue{Int64}(), d = DataValue{Float64}(4.0), e = DataValue{Int64}(), f = DataValue{Float64}(5.0), a = DataValue{Int64}(1)) 
 (b = DataValue{Float64}(3.0), c = DataValue{Int64}(3), d = DataValue{Float64}(4.0), e = DataValue{Int64}(4), f = DataValue{Float64}(5.0), a = DataValue{Int64}())
 (b = DataValue{Float64}(), c = DataValue{Int64}(3), d = DataValue{Float64}(), e = DataValue{Int64}(4), f = DataValue{Float64}(5.0), a = DataValue{Int64}(8))     
 (b = DataValue{Float64}(), c = DataValue{Int64}(3), d = DataValue{Float64}(4.0), e = DataValue{Int64}(4), f = DataValue{Float64}(5.0), a = DataValue{Int64}())   

Now lets benchmark these:

In [8]:
@benchmark ((b=i.b,c=i.c,d=i.d,e=i.e,f=i.f,a=i.a) for i in A) |> collect

BenchmarkTools.Trial: 
  memory estimate:  1.13 KiB
  allocs estimate:  26
  --------------
  minimum time:     2.975 μs (0.00% GC)
  median time:      3.288 μs (0.00% GC)
  mean time:        4.432 μs (18.14% GC)
  maximum time:     5.559 ms (99.88% GC)
  --------------
  samples:          10000
  evals/sample:     8

In [9]:
@benchmark ((b=i.b,c=i.c,d=i.d,e=i.e,f=i.f,a=i.a) for i in B) |> collect

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  2
  --------------
  minimum time:     105.471 ns (0.00% GC)
  median time:      108.262 ns (0.00% GC)
  mean time:        133.352 ns (11.17% GC)
  maximum time:     46.822 μs (99.69% GC)
  --------------
  samples:          10000
  evals/sample:     932

The problem here is simple: the `Missing` case is more than an order of magnitude slower.

## Second order problem 1: sink array type is not great

If we look at the type of the vector that we generated for the two cases, we see this for the `Missing` case:

In [10]:
A2

4-element Array{NamedTuple{(:b, :c, :d, :e, :f, :a),T} where T<:Tuple,1}:
 (b = 2.0, c = missing, d = 4.0, e = missing, f = 5.0, a = 1)
 (b = 3.0, c = 3, d = 4.0, e = 4, f = 5.0, a = missing)      
 (b = missing, c = 3, d = missing, e = 4, f = 5.0, a = 8)    
 (b = missing, c = 3, d = 4.0, e = 4, f = 5.0, a = missing)  

And this for the `DataValue` case:

In [11]:
B2

4-element Array{NamedTuple{(:b, :c, :d, :e, :f, :a),Tuple{DataValue{Float64},DataValue{Int64},DataValue{Float64},DataValue{Int64},DataValue{Float64},DataValue{Int64}}},1}:
 (b = DataValue{Float64}(2.0), c = DataValue{Int64}(), d = DataValue{Float64}(4.0), e = DataValue{Int64}(), f = DataValue{Float64}(5.0), a = DataValue{Int64}(1)) 
 (b = DataValue{Float64}(3.0), c = DataValue{Int64}(3), d = DataValue{Float64}(4.0), e = DataValue{Int64}(4), f = DataValue{Float64}(5.0), a = DataValue{Int64}())
 (b = DataValue{Float64}(), c = DataValue{Int64}(3), d = DataValue{Float64}(), e = DataValue{Int64}(4), f = DataValue{Float64}(5.0), a = DataValue{Int64}(8))     
 (b = DataValue{Float64}(), c = DataValue{Int64}(3), d = DataValue{Float64}(4.0), e = DataValue{Int64}(4), f = DataValue{Float64}(5.0), a = DataValue{Int64}())   

The problem here is that the element type of `A2` (the `Missing` case) is not concrete, so individual elements in this array are boxed and stored on the heap, which is not good. For the `DataValue` case `B2` the element type is concrete and so we should be getting a much better array layout in memory.

## Second order problem 2: all information about the potential missingnes of column `f` is lost

If we look at column `f` in the output for the `Missing` case (`A2`), there is no way to recover the information that column `f` can actually hold missing values. If we look at the element types of each row, we see this:

In [12]:
[typeof(i) for i in A2]

4-element Array{DataType,1}:
 NamedTuple{(:b, :c, :d, :e, :f, :a),Tuple{Float64,Missing,Float64,Missing,Float64,Int64}}
 NamedTuple{(:b, :c, :d, :e, :f, :a),Tuple{Float64,Int64,Float64,Int64,Float64,Missing}}  
 NamedTuple{(:b, :c, :d, :e, :f, :a),Tuple{Missing,Int64,Missing,Int64,Float64,Int64}}    
 NamedTuple{(:b, :c, :d, :e, :f, :a),Tuple{Missing,Int64,Float64,Int64,Float64,Missing}}  

So from that we can't recover the info that `f` could have held a missing value, and from the type of the entire array of course also not:

In [13]:
typeof(A2)

Array{NamedTuple{(:b, :c, :d, :e, :f, :a),T} where T<:Tuple,1}

This information is simply entirely gone and lost!

For the `DataValue` case `B2` that is not so, just looking at the array we can see that `f` is a column that could have held missing values:

In [14]:
typeof(B2)

Array{NamedTuple{(:b, :c, :d, :e, :f, :a),Tuple{DataValue{Float64},DataValue{Int64},DataValue{Float64},DataValue{Int64},DataValue{Float64},DataValue{Int64}}},1}

So if we for example pipe the result into a `DataFrame`, we get what we want, namely that the column `f` can have missing values:

In [15]:
DataFrame(B2)

,b,c,d,e,f,a
,Float64⍰,Int64⍰,Float64⍰,Int64⍰,Float64⍰,Int64⍰
1,2.0,missing,4.0,missing,5.0,1
2,3.0,3,4.0,4,5.0,missing
3,missing,3,missing,4,5.0,8
4,missing,3,4.0,4,5.0,missing


(I'm not showing the same conversion of `A2` to a `DataFrame` because all columns just have the `Any` type there).

One question now is whether this potential missingness information should propagate through this projection, if there are actually no mising values in the `f` column in the source data. My main argument why it should is that a) that is what SQL does, and b) I think there is a strong analogy with say different number types. Assume we have a table with one column of type `Float64`:

In [16]:
C = [(a=4.,), (a=5.,), (a=8.,)]

3-element Array{NamedTuple{(:a,),Tuple{Float64}},1}:
 (a = 4.0,)
 (a = 5.0,)
 (a = 8.0,)

If we run a generator over that

In [17]:
((a=i.a,) for i in C) |> collect

3-element Array{NamedTuple{(:a,),Tuple{Float64}},1}:
 (a = 4.0,)
 (a = 5.0,)
 (a = 8.0,)

we wouldn't want the fields here to become `Int`s, even though all the values that we have in the source data are of course integers. I think in the same way we would want potential missinginess to propagate: even if a source doesn't have any missing values in a column, but the column is of a type that could hold missing values, that property should propagate through.